In [ ]:
!pip install tflearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 KB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=f1eef94a8c8e4b142ab72473d4d599bfcf17bab970a8cfbedcec63b9de921c09
  Stored in directory: /root/.cache/pip/wheels/65/9b/15/cb1e6b279c14ed897530d15cfd7da8e3df8a947e593f5cfe59
Successfully built tflearn


In [ ]:
import numpy as np
import nltk, string
import dill
import pickle
import torch
import tflearn
import json
import os
import re
from sklearn.metrics.pairwise import cosine_similarity


nltk.download('punkt') # if necessary...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
PATH = os.path.dirname(os.path.realpath('__file__'))

In [ ]:
stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

list_words = []
labels = []
docs_x = [] 
docs_y = [] 

def cleanText(Text):
    cleanedText = Text.replace('\n',' ').lower()
    cleanedText = cleanedText.replace("\xc2\xa0", "")
    cleanedText = re.sub('([,])','', cleanedText)
    cleanedText = re.sub(' +',' ', cleanedText)
    cleanedText = cleanedText.encode('ascii', 'ignore').decode('ascii')
    return cleanedText

def getData():
    listTexts = source_data['abstract'].tolist()
    finallist=[]
    print(listTexts)
    for i in range (0,4):
        textDictionary = {"tag":i}
        listTexts_i = cleanText(listTexts[i])
        print(listTexts_i)
        textDictionary.update(texts = listTexts_i)
        finallist.append(textDictionary)
    finalDictionary={"intents":finallist}       
    return finalDictionary


combinedDictionary = dict()
print ('Getting Text Data')
combinedDictionary.update(getData())
print ('Total len of dictionary', len(combinedDictionary))

print ('Saving text data dictionary')
np.save(PATH + '/textDictionary.npy', combinedDictionary)

with open(PATH + '/file.txt', 'w') as file:
     file.write(json.dumps(combinedDictionary))



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
with open(PATH + "/file.txt") as file:
    dataset = json.load(file)
    
for intent in dataset["intents"]:
    for pattern in intent["texts"]: 
        split_words = nltk.word_tokenize(pattern) 
        list_words.extend(split_words) 
        docs_x.append(split_words) 
        docs_y.append(intent["tag"]) 
    if intent["tag"] not in labels:
        labels.append(intent["tag"]) 

list_words = [stemmer.stem(w.lower()) for w in list_words if w != "?"] 
list_words = sorted(list(set(list_words))) 

labels = sorted(labels)

training = [] 
output = [] 

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = [] 

    split_words = [stemmer.stem(w.lower()) for w in doc] 

    for w in list_words:
        if w in split_words:
            bag.append(1) 
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1 

    training.append(bag) 
    output.append(output_row) 

In [ ]:
def bag_of_words(s, list_words): 
    bag = [0 for _ in range(len(list_words))]

    inp_str_words = nltk.word_tokenize(s)
    inp_str_words = [stemmer.stem(word.lower()) for word in inp_str_words]

    for search_element in inp_str_words:
        for i, w in enumerate(list_words):
            if w == search_element:
                bag[i] = 1 
            
    return np.array(bag)

In [ ]:
training = np.array(training) 
output = np.array(output)

net = tflearn.input_data(shape=[None, len(training[0])]) 
net = tflearn.fully_connected(net, 8) #Hidden layer with 8 neurons.
net = tflearn.fully_connected(net, 8) #Another hidden layer with 8 neurons.
net = tflearn.fully_connected(net, 8) #Another hidden layer with 8 neurons.
net = tflearn.fully_connected(net, len(output[0]), activation="softmax") 

model = tflearn.DNN(net) 
try:
    model.predict(PATH + "/model.tflearn")
    model.load()
except:
    model.fit(training, output, n_epoch=2000, batch_size=8, show_metric=True)
    model.save(PATH + "/model.tflearn") 

Training Step: 7437  | total loss: 1.30984 | time: 0.929s
| Adam | epoch: 014 | loss: 1.30984 - acc: 0.2777 -- iter: 1984/4423


In [ ]:
def cosine_similarity(text1, text2):
    tfidf = model.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

    
def is_plagiarism(similarity_score, plagiarism_threshold):

  is_plagiarism = False

  if(similarity_score >= plagiarism_threshold):
    is_plagiarism = True

  return is_plagiarism

def run_plagiarism_analysis(query_text, data, plagiarism_threshold=0.8):

    top_N=3

    # Run similarity Search
    data["similarity"] = data["abstract"].apply(lambda x: cosine_similarity(query_text, x))

    similar_articles = data.sort_values(by='similarity', ascending=False)[0:top_N+1]
    formated_result = similar_articles[["abstract", "similarity"]].reset_index(drop = True)
    similarity_score = formated_result.iloc[0]["similarity"] 
    similarity_percentage = str(round(similarity_score *100)) + '%'
    most_similar_article = formated_result.iloc[0]["abstract"] 
    is_plagiarism_bool = is_plagiarism(similarity_score, plagiarism_threshold)


    plagiarism_decision = {'similarity_score': similarity_score, 
                           'similarity_percentage': similarity_percentage,
                            'is_plagiarism': is_plagiarism_bool,
                            'most_similar_article': most_similar_article, 
                            'article_submitted': query_text
                        }
    #return plagiarism_decision
    return formated_result

In [ ]:
new_incoming_text = "hi, hello, how are you?"

# Run the plagiarism detection
analysis_result = run_plagiarism_analysis(new_incoming_text, source_data, plagiarism_threshold=0.8)
analysis_result

{'similarity_score': 0.22576484600261604,
 'similarity_percentage': '23%',
 'is_plagiarism': False,
 'most_similar_article': 'How safe are our microbiology labs?',
 'article_submitted': 'hi, hello, how are you?'}

In [ ]:
new_incoming_text =  source_data.iloc[1]['abstract']

# Run the plagiarism detection
analysis_result = run_plagiarism_analysis(new_incoming_text, source_data, plagiarism_threshold=0.8)
analysis_result

{'similarity_score': 1.0000000000000004,
 'similarity_percentage': '100%',
 'is_plagiarism': True,
 'most_similar_article': 'A critical aspect of reliable communication involves the design of codes that\nallow transmissions to be robustly and computationally efficiently decoded\nunder noisy conditions. Advances in the design of reliable codes have been\ndriven by coding theory and have been sporadic. Recently, it is shown that\nchannel codes that are comparable to modern codes can be learned solely via\ndeep learning. In particular, Turbo Autoencoder (TURBOAE), introduced by Jiang\net al., is shown to achieve the reliability of Turbo codes for Additive White\nGaussian Noise channels. In this paper, we focus on applying the idea of\nTURBOAE to various practical channels, such as fading channels and chirp noise\nchannels. We introduce TURBOAE-TI, a novel neural architecture that combines\nTURBOAE with a trainable interleaver design. We develop a carefully-designed\ntraining procedure and

In [ ]:
pickle.dump(run_plagiarism_analysis, open('similarity_model.pkl','wb'))
g = pickle.load(open('similarity_model.pkl','rb'))
g("possible further increases in nucleic acid", source_data, plagiarism_threshold=0.8)

{'similarity_score': 0.18381708471385255,
 'similarity_percentage': '18%',
 'is_plagiarism': False,
 'most_similar_article': 'BACKGROUND Nucleic acid test and antibody assay have been employed in the diagnosis for SARS-CoV-2 infection, but the use of viral antigen for diagnosis has not been successfully developed. Theoretically, viral antigen is the specific marker of the virus and precedes antibody appearance within the infected population. There is a clear need of detection of viral antigen for rapid and early diagnosis. METHODS We included a cohort of 239 participants with suspected SARS-CoV-2 infection from 7 centers for the study. We measured nucleocapsid protein in nasopharyngeal swab samples in parallel with the nucleic acid test. Nucleic acid test was taken as the reference standard, and statistical evaluation was taken in blind. We detected nucleocapsid protein in 20 urine samples in another center, employing nasopharyngeal swab nucleic acid test as reference standard. RESULTS